In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import  load_model
from keras.layers import  RepeatVector, TimeDistributed, Activation
from sklearn.preprocessing import MinMaxScaler
import joblib
# Verificar si se detecta una GPU
if tf.test.is_gpu_available():
    print("Se detectó una GPU. TensorFlow utilizará la GPU para acelerar el entrenamiento y la predicción.")
else:
    print("No se detectó una GPU. TensorFlow utilizará la CPU para el entrenamiento y la predicción.")

# El resto del código sigue igual...


2023-07-25 15:28:25.045845: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
No se detectó una GPU. TensorFlow utilizará la CPU para el entrenamiento y la predicción.


In [4]:
import pandas as pd

df_lima = pd.read_csv("./TimeSeriesDataLIMAAGUASPROFUNDAS.csv")
df_lima.columns = ["fecha", "temperatura"]  # Sin doble corchete en el nombre de las columnas

# Convertir la columna "fecha" al formato de fecha
df_lima["fecha"] = pd.to_datetime(df_lima["fecha"])

# Definir las fechas de inicio y fin para el rango que quieres seleccionar
fecha_inicio = pd.to_datetime("1981-01-01")
fecha_final = pd.to_datetime("2023-06-01")

# Seleccionar el rango de fechas
df_limaR=df_lima
df_lima = df_lima[(df_lima["fecha"] >= fecha_inicio) & (df_lima["fecha"] < fecha_final)]

df = df_lima  # El DataFrame final es df
df

,fecha,temperatura
0,1981-09-01,14.98
1,1981-09-02,13.85
2,1981-09-03,11.82
3,1981-09-04,12.37
4,1981-09-05,16.15
...,...,...
15243,2023-05-27,21.12
15244,2023-05-28,21.22
15245,2023-05-29,21.79
15246,2023-05-30,21.59


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Definir el número de días para la predicción
dias_prediccion = 120



# Función para generar datos aleatorios simulados de temperatura superficial del mar en Perú reemplazado por el df 
df

# Función para preparar los datos para el entrenamiento de la red neuronal
def prepare_data(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back - 1):
        X.append(data[i:(i + look_back)])
        y.append(data[i + look_back])
    return np.array(X), np.array(y)

look_back = 30
X, y = prepare_data(df['temperatura'].values, look_back)

# Redimensionar los datos para LSTM (número de muestras, pasos de tiempo, características)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Construir el modelo de LSTM
model = Sequential()
model.add(LSTM(50, input_shape=(look_back, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X, y, epochs=10, batch_size=1, verbose=2)


# Realizar la predicción 
ultimos_dias = np.copy(df['temperatura'].values[-look_back:])
predicciones = []
for _ in range(dias_prediccion):
    entrada = ultimos_dias[-look_back:]
    entrada = np.reshape(entrada, (1, look_back, 1))
    prediccion = model.predict(entrada)[0][0]
    predicciones.append(prediccion)
    ultimos_dias = np.append(ultimos_dias, prediccion)

# Crear DataFrame con las predicciones
fecha_inicio_prediccion = df['fecha'].iloc[-1] + pd.Timedelta(days=1)
fecha_final_prediccion = fecha_inicio_prediccion + pd.Timedelta(days=dias_prediccion - 1)
fechas_prediccion = pd.date_range(start=fecha_inicio_prediccion, end=fecha_final_prediccion, freq='D')
df_prediccion = pd.DataFrame({'fecha': fechas_prediccion, 'temperatura': predicciones})

# Graficar los datos originales y las predicciones
plt.figure(figsize=(12, 6))
plt.plot(df['fecha'], df['temperatura'], label='Temperatura Real')
plt.plot(df_prediccion['fecha'], df_prediccion['temperatura'], label='Predicción')
plt.xlabel('Fecha')
plt.ylabel('Temperatura (°C)')
plt.legend()
plt.title('Predicción de Temperatura Superficial del Mar en Perú')
plt.grid(True)
plt.show()


In [2]:
model = load_model('modelo_lstm_entrenado.h5')


2023-07-25 15:28:39.831489: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-25 15:28:39.832969: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-25 15:28:39.833704: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [5]:
# Definir el número de días para la predicción
dias_prediccion = 120

# Realizar la predicción para el próximo mes
ultimos_dias = np.copy(df['temperatura'].values[-look_back:])
predicciones = []
for _ in range(dias_prediccion):
    entrada = ultimos_dias[-look_back:]
    entrada = np.reshape(entrada, (1, look_back, 1))
    prediccion = model.predict(entrada)[0][0]
    predicciones.append(prediccion)
    ultimos_dias = np.append(ultimos_dias, prediccion)

# Crear DataFrame con las predicciones
fecha_inicio_prediccion = df['fecha'].iloc[-1] + pd.Timedelta(days=1)
fecha_final_prediccion = fecha_inicio_prediccion + pd.Timedelta(days=dias_prediccion - 1)
fechas_prediccion = pd.date_range(start=fecha_inicio_prediccion, end=fecha_final_prediccion, freq='D')
df_prediccion = pd.DataFrame({'fecha': fechas_prediccion, 'temperatura': predicciones})

# Graficar los datos originales y las predicciones
plt.figure(figsize=(12, 6))
plt.plot(df['fecha'], df['temperatura'], label='Temperatura Real')
plt.xlim(pd.to_datetime('01-01-2017'),pd.to_datetime('12-31-2023'))
plt.plot(df_limaR['fecha'], df_limaR['temperatura'], label='Temperatura Real lima')
plt.plot(df_prediccion['fecha'], df_prediccion['temperatura'], label='Predicción')
plt.xlabel('Fecha')
plt.ylabel('Temperatura (°C)')
plt.legend()
plt.title('Predicción de Temperatura Superficial del Mar en Perú')
plt.grid(True)
plt.show()

NameError: name 'look_back' is not defined